In [1]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Utils

In [3]:
CLASS_NAME_TO_ID = {'Unformed': 0, 'Burr': 1}
CLASS_ID_TO_NAME = {0: 'Unformed', 1: 'Burr'}
BOX_COLOR = {'Unformed':(200, 0, 0), 'Burr':(0, 0, 200)}
TEXT_COLOR = (255, 255, 255)

def save_model(model_state, model_name, save_dir="./trained_model"):
    os.makedirs(save_dir, exist_ok=True)
    torch.save(model_state, os.path.join(save_dir, model_name))


def visualize_bbox(image, bbox, class_name, color=BOX_COLOR, thickness=2):
    x_center, y_center, w, h = bbox
    x_min = int(x_center - w/2)
    y_min = int(y_center - h/2)
    x_max = int(x_center + w/2)
    y_max = int(y_center + h/2)
    
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color[class_name], thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(image, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), color[class_name], -1)
    cv2.putText(
        image,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return image


def visualize(image, bboxes, category_ids):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
#         print('category_id: ',category_id)
        class_name = CLASS_ID_TO_NAME[category_id.item()]
        img = visualize_bbox(img, bbox, class_name)
    return img

## Datasets

In [4]:
class PET_dataset():
    def __init__(self,part,neck_dir,body_dir,phase, transformer=None, aug=None, aug_factor=0):
        self.neck_dir=neck_dir
        self.body_dir=body_dir
        self.part=part
        self.phase=phase
        self.transformer=transformer
        self.aug=aug
        self.aug_factor=aug_factor
        if(self.part=="body"):
            self.image_files = sorted([fn for fn in os.listdir(self.body_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.body_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        elif(self.part=="neck"):
            self.image_files = sorted([fn for fn in os.listdir(self.neck_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.neck_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        
        self.auged_img_list, self.auged_label_list=self.make_aug_list(self.image_files, self.label_files)
        
    def __getitem__(self,index):
        if(self.aug==None):
            filename, image = self.get_image(self.part, index)
            bboxes, class_ids = self.get_label(self.part, index)

            if(self.transformer):
                transformed_data=self.transformer(image=image, bboxes=bboxes, class_ids=class_ids)
                image = transformed_data['image']
                bboxes = np.array(transformed_data['bboxes'])
                class_ids = np.array(transformed_data['class_ids'])


            target = {}
    #         print(f'bboxes:{bboxes}\nclass_ids:{class_ids}\nlen_bboxes:{len(bboxes)}\nlen_class_ids:{len(class_ids)}')
    #         print(f'filename: {filename}')
            target["boxes"] = torch.Tensor(bboxes).float()
            target["labels"] = torch.Tensor(class_ids).long()

            ###
            bboxes=torch.Tensor(bboxes).float()
            class_ids=torch.Tensor(class_ids).long()
            target = np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1)
            ###
        else:
            image=self.auged_img_list[index][1]
            target=self.auged_label_list[index]
            filename=self.auged_img_list[index][0]
        return image, target, filename
    
    def __len__(self, ):
        length=0
        if(self.aug==None):
            length=len(self.image_files)
        else:
            length=len(self.auged_img_list)
        return length
    
    def make_aug_list(self,ori_image_list,ori_label_files):
        aug_image_list=[]
        aug_label_list=[]
        
        print(f"start making augmented images-- augmented factor:{self.aug_factor}")
        for i in range(len(ori_image_list)):
            filename, ori_image = self.get_image(self.part, i)
            ori_bboxes, ori_class_ids = self.get_label(self.part, i)
#             print(f"{filename}--------------\n{ori_bboxes}")
            for j in range(self.aug_factor):
                auged_data=self.aug(image=ori_image, bboxes=ori_bboxes, class_ids=ori_class_ids)
                image = auged_data['image']
                bboxes = np.array(auged_data['bboxes'])
                class_ids = np.array(auged_data['class_ids'])
                
                bboxes=torch.Tensor(bboxes).float()
                class_ids=torch.Tensor(class_ids).long()
                
                aug_image_list.append((filename, image))
                aug_label_list.append(np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1))
        
        print(f"total length of augmented images: {len(aug_image_list)}")
        
        return aug_image_list, aug_label_list
        
    
    def get_image(self, part, index): # 이미지 불러오는 함수
        filename = self.image_files[index]
        if(part=="body"):
#             print(f"body called!-> {self.part}")
            image_path = self.body_dir+"/"+self.phase+"/image/"+filename
        elif(part=="neck"):
#             print(f"neck called!-> {self.part}")
            image_path = self.neck_dir+"/"+self.phase+"/image/"+filename
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return filename, image
    
    def get_label(self, part, index): # label (box좌표, class_id) 불러오는 함수
        label_filename=self.label_files[index]
        if(part=="body"):
#             print(f"body label called!-> {self.part}")
            label_path = self.body_dir+"/"+self.phase+"/label/"+label_filename
        elif(part=="neck"):
#             print(f"neck label called!-> {self.part}")
            label_path = self.neck_dir+"/"+self.phase+"/label/"+label_filename
        with open(label_path, 'r') as file:
            labels = file.readlines()
        
        class_ids=[]
        bboxes=[]
        for label in labels:
            label=label.replace("\n", "")
            obj=label.split(' ')[0]
            coor=label.split(' ')[1:]
            obj=int(obj)
            coor=list(map(float, coor))
            class_ids.append(obj)
            bboxes.append(coor)
            
        return bboxes, class_ids
    

In [5]:
IMAGE_SIZE = 448

transformer = A.Compose([ 
        # bounding box의 변환, augmentation에서 albumentations는 Detection 학습을 할 때 굉장히 유용하다. 
        A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        # albumentations 라이브러리에서는 Normalization을 먼저 진행해 주고 tensor화를 진행해 주어야한다.
    ],
    # box 위치에 대한 transformation도 함께 진행된다. 
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

augmentator=A.Compose([
#     A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
#     A.HorizontalFlip(p=0.7),
#     A.Sharpen(p=0.7),
    A.RandomSizedBBoxSafeCrop(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1),
#     A.VerticalFlip (p=0.5),
#     A.HueSaturationValue(p=0.5),
#     A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
#     A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
#     ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

def collate_fn(batch):
    image_list = []
    target_list = []
    filename_list = []
    
    for a,b,c in batch:
        image_list.append(a)
        target_list.append(b)
        filename_list.append(c)

    return torch.stack(image_list, dim=0), target_list, filename_list


In [6]:
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'

trainset_crop=PET_dataset(part='neck',neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='train', transformer=transformer, aug=augmentator, aug_factor=50)



start making augmented images-- augmented factor:50
total length of augmented images: 10500


In [6]:
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'
PHASE='train'
AUG_FAC=50
trainset_crop=PET_dataset(part='neck',neck_dir=NECK_PATH,body_dir=BODY_PATH,phase=PHASE, transformer=transformer, aug=augmentator, aug_factor=AUG_FAC)


_, _, temp_filename=trainset_crop[0]
file_num=0
for i in range(len(trainset_crop)):
    image, target, filename = trainset_crop[i]
    if(temp_filename==filename):
        file_num=file_num+1
    else:
        file_num=1
    temp_filename=filename
    file_title=filename.split(".")[0]
    save_title=f"{file_title}_aug{file_num}"
    
    #이미지 저장
    cv2.imwrite(f"/home/host_data/PET_data_image_patching/patched_Neck/{PHASE}/image/"+save_title+".jpg", image)
        
    #라벨파일 저장
    f = open(f"/home/host_data/PET_data_image_patching/patched_Neck/{PHASE}/label/"+save_title+".txt", 'w')
    print(f"ori-tar:{target}")
    for j in range(len(target)):
        write_str=f"{int(target[j][4])} {target[j][0]} {target[j][1]} {target[j][2]} {target[j][3]}\n"
        print(write_str)
        f.write(write_str)

    print(save_title)
#     print(str(target).replace("[", " ").replace("]", " "))
    print(target)

start making augmented images-- augmented factor:50
total length of augmented images: 10500
ori-tar:[[0.85556775 0.78571415 0.06380518 0.27142885 0.        ]]
0 0.8555677533149719 0.7857141494750977 0.06380517780780792 0.2714288532733917

shape1_10_aug1
[[0.85556775 0.78571415 0.06380518 0.27142885 0.        ]]
ori-tar:[[0.84321934 0.84803915 0.09322044 0.18627471 0.        ]]
0 0.8432193398475647 0.8480391502380371 0.09322044253349304 0.1862747073173523

shape1_10_aug2
[[0.84321934 0.84803915 0.09322044 0.18627471 0.        ]]
ori-tar:[[0.95944381 0.35416651 0.06111118 0.39583376 0.        ]]
0 0.9594438076019287 0.35416650772094727 0.06111117824912071 0.3958337604999542

shape1_10_aug3
[[0.95944381 0.35416651 0.06111118 0.39583376 0.        ]]
ori-tar:[[0.93171018 0.76404488 0.08634233 0.14232224 0.        ]]
0 0.9317101836204529 0.7640448808670044 0.08634232729673386 0.14232224225997925

shape1_10_aug4
[[0.93171018 0.76404488 0.08634233 0.14232224 0.        ]]
ori-tar:[[0.9034723  0

ori-tar:[[0.1495335  0.62448949 0.08971903 0.16326559 0.        ]
 [0.15140112 0.33877498 0.07850415 0.13877511 0.        ]]
0 0.1495334953069687 0.6244894862174988 0.0897190272808075 0.16326558589935303

0 0.1514011174440384 0.33877497911453247 0.07850415259599686 0.13877511024475098

shape1_108_aug22
[[0.1495335  0.62448949 0.08971903 0.16326559 0.        ]
 [0.15140112 0.33877498 0.07850415 0.13877511 0.        ]]
ori-tar:[[0.54980135 0.56020898 0.0637446  0.20942445 0.        ]
 [0.55112827 0.1937166  0.05577652 0.17800996 0.        ]]
0 0.5498013496398926 0.5602089762687683 0.06374459713697433 0.2094244509935379

0 0.5511282682418823 0.1937166005373001 0.05577652156352997 0.17800995707511902

shape1_108_aug23
[[0.54980135 0.56020898 0.0637446  0.20942445 0.        ]
 [0.55112827 0.1937166  0.05577652 0.17800996 0.        ]]
ori-tar:[[0.28682333 0.5932833  0.18604527 0.14925399 0.        ]
 [0.29069611 0.33208907 0.16278961 0.12686531 0.        ]]
0 0.2868233323097229 0.59328329563

ori-tar:[[0.16565409 0.57677138 0.09118599 0.14566956 0.        ]]
0 0.16565409302711487 0.5767713785171509 0.09118598699569702 0.14566956460475922

shape1_114_aug45
[[0.16565409 0.57677138 0.09118599 0.14566956 0.        ]]
ori-tar:[[0.43910319 0.8152169  0.09615446 0.26811644 0.        ]]
0 0.4391031861305237 0.8152168989181519 0.09615445882081985 0.26811644434928894

shape1_114_aug46
[[0.43910319 0.8152169  0.09615446 0.26811644 0.        ]]
ori-tar:[[0.86593491 0.7653057  0.13186897 0.25170115 0.        ]]
0 0.8659349083900452 0.7653056979179382 0.13186897337436676 0.25170114636421204

shape1_114_aug47
[[0.86593491 0.7653057  0.13186897 0.25170115 0.        ]]
ori-tar:[[0.21112993 0.55952328 0.09820029 0.29365134 0.        ]]
0 0.21112993359565735 0.5595232844352722 0.09820029139518738 0.2936513423919678

shape1_114_aug48
[[0.21112993 0.55952328 0.09820029 0.29365134 0.        ]]
ori-tar:[[0.30927879 0.5037874  0.06872896 0.28030354 0.        ]]
0 0.30927878618240356 0.503787398338

ori-tar:[[0.30720648 0.68518466 0.08468573 0.17592555 0.        ]]
0 0.30720648169517517 0.6851846575737 0.08468572795391083 0.17592555284500122

shape1_128_aug20
[[0.30720648 0.68518466 0.08468573 0.17592555 0.        ]]
ori-tar:[[0.08677649 0.6300571  0.0431594  0.21965271 0.        ]]
0 0.0867764875292778 0.6300570964813232 0.04315939545631409 0.2196527123451233

shape1_128_aug21
[[0.08677649 0.6300571  0.0431594  0.21965271 0.        ]]
ori-tar:[[0.14239043 0.51785499 0.10217518 0.67856997 0.        ]]
0 0.1423904299736023 0.5178549885749817 0.10217517614364624 0.6785699725151062

shape1_128_aug22
[[0.14239043 0.51785499 0.10217518 0.67856997 0.        ]]
ori-tar:[[0.21595173 0.66264915 0.18288164 0.45783037 0.        ]]
0 0.21595172584056854 0.6626491546630859 0.18288163840770721 0.45783036947250366

shape1_128_aug23
[[0.21595173 0.66264915 0.18288164 0.45783037 0.        ]]
ori-tar:[[0.64549017 0.39534745 0.19262533 0.44185954 0.        ]]
0 0.6454901695251465 0.3953474462032318 

ori-tar:[[0.12915076 0.84883678 0.16974248 0.20930298 0.        ]]
0 0.12915076315402985 0.8488367795944214 0.16974247992038727 0.20930297672748566

shape1_133_aug50
[[0.12915076 0.84883678 0.16974248 0.20930298 0.        ]]
ori-tar:[[0.91734403 0.89356482 0.06504112 0.18316875 0.        ]]
0 0.9173440337181091 0.8935648202896118 0.06504112482070923 0.18316875398159027

shape1_134_aug1
[[0.91734403 0.89356482 0.06504112 0.18316875 0.        ]]
ori-tar:[[0.62592548 0.75423783 0.17777908 0.20904005 0.        ]]
0 0.6259254813194275 0.7542378306388855 0.17777907848358154 0.20904004573822021

shape1_134_aug2
[[0.62592548 0.75423783 0.17777908 0.20904005 0.        ]]
ori-tar:[[0.8589853  0.83209026 0.04127287 0.27612007 0.        ]]
0 0.8589853048324585 0.8320902585983276 0.04127287492156029 0.27612006664276123

shape1_134_aug3
[[0.8589853  0.83209026 0.04127287 0.27612007 0.        ]]
ori-tar:[[0.82370508 0.69615459 0.04780912 0.28461605 0.        ]]
0 0.8237050771713257 0.6961545944213867

...

ori-tar:[[0.47956663 0.21428593 0.16586414 0.10895883 0.        ]]
0 0.47956663370132446 0.2142859250307083 0.16586413979530334 0.10895882546901703

shape5_7_aug10
[[0.47956663 0.21428593 0.16586414 0.10895883 0.        ]]
ori-tar:[[0.68020642 0.39583373 0.10191947 0.20833331 0.        ]]
0 0.6802064180374146 0.39583373069763184 0.10191947221755981 0.2083333134651184

shape5_7_aug11
[[0.68020642 0.39583373 0.10191947 0.20833331 0.        ]]
ori-tar:[[0.86175364 0.21513379 0.08404322 0.13353114 0.        ]]
0 0.8617536425590515 0.21513378620147705 0.0840432196855545 0.1335311383008957

shape5_7_aug12
[[0.86175364 0.21513379 0.08404322 0.13353114 0.        ]]
ori-tar:[[0.75257111 0.24635068 0.07885655 0.16423357 0.        ]]
0 0.7525711059570312 0.24635067582130432 0.07885655015707016 0.16423356533050537

shape5_7_aug13
[[0.75257111 0.24635068 0.07885655 0.16423357 0.        ]]
ori-tar:[[0.90426666 0.15502816 0.07549177 0.12569831 0.        ]]
0 0.9042666554450989 0.15502816438674927 0.0

ori-tar:[[0.86937886 0.70432729 0.08136987 0.18749994 0.        ]]
0 0.8693788647651672 0.7043272852897644 0.08136986941099167 0.18749994039535522

shape6_11_aug37
[[0.86937886 0.70432729 0.08136987 0.18749994 0.        ]]
ori-tar:[[0.95243013 0.61259574 0.03929651 0.14885493 0.        ]]
0 0.9524301290512085 0.6125957369804382 0.03929651156067848 0.14885492622852325

shape6_11_aug38
[[0.95243013 0.61259574 0.03929651 0.14885493 0.        ]]
ori-tar:[[0.86695266 0.55916059 0.08154448 0.14885493 0.        ]]
0 0.866952657699585 0.559160590171814 0.08154448121786118 0.14885492622852325

shape6_11_aug39
[[0.86695266 0.55916059 0.08154448 0.14885493 0.        ]]
ori-tar:[[0.88163877 0.71474409 0.05766271 0.24999993 0.        ]]
0 0.881638765335083 0.7147440910339355 0.0576627142727375 0.24999992549419403

shape6_11_aug40
[[0.88163877 0.71474409 0.05766271 0.24999993 0.        ]]
ori-tar:[[0.91277885 0.50178599 0.03853928 0.13928567 0.        ]]
0 0.9127788543701172 0.5017859935760498 0.038

ori-tar:[[0.82681549 0.50163925 0.10055813 0.13114762 0.        ]]
0 0.8268154859542847 0.5016392469406128 0.10055813193321228 0.1311476230621338

shape6_17_aug16
[[0.82681549 0.50163925 0.10055813 0.13114762 0.        ]]
ori-tar:[[0.86115688 0.67415714 0.08925573 0.22471924 0.        ]]
0 0.8611568808555603 0.6741571426391602 0.08925572782754898 0.22471924126148224

shape6_17_aug17
[[0.86115688 0.67415714 0.08925573 0.22471924 0.        ]]
ori-tar:[[0.61846131 0.53237391 0.16615297 0.28776997 0.        ]]
0 0.6184613108634949 0.5323739051818848 0.1661529690027237 0.28776997327804565

shape6_17_aug18
[[0.61846131 0.53237391 0.16615297 0.28776997 0.        ]]
ori-tar:[[0.62941158 0.5363127  0.15882269 0.22346383 0.        ]]
0 0.6294115781784058 0.5363126993179321 0.1588226854801178 0.22346383333206177

shape6_17_aug19
[[0.62941158 0.5363127  0.15882269 0.22346383 0.        ]]
ori-tar:[[0.88297862 0.61702102 0.06382945 0.42553219 0.        ]]
0 0.882978618144989 0.6170210242271423 0.063

ori-tar:[[0.95784479 0.70338994 0.0351291  0.59322011 0.        ]]
0 0.9578447937965393 0.7033899426460266 0.03512910380959511 0.5932201147079468

shape6_22_aug39
[[0.95784479 0.70338994 0.0351291  0.59322011 0.        ]]
ori-tar:[[0.97004205 0.21572606 0.04279637 0.14112943 0.        ]]
0 0.9700420498847961 0.2157260626554489 0.04279636964201927 0.14112943410873413

shape6_22_aug40
[[0.97004205 0.21572606 0.04279637 0.14112943 0.        ]]
ori-tar:[[0.97416908 0.33388725 0.03690068 0.1162794  0.        ]]
0 0.9741690754890442 0.3338872492313385 0.036900684237480164 0.11627940088510513

shape6_22_aug41
[[0.97416908 0.33388725 0.03690068 0.1162794  0.        ]]
ori-tar:[[0.66666031 0.2553767  0.35714591 0.18817258 0.        ]]
0 0.6666603088378906 0.2553766965866089 0.35714590549468994 0.1881725788116455

shape6_22_aug42
[[0.66666031 0.2553767  0.35714591 0.18817258 0.        ]]
ori-tar:[[0.84359938 0.47535256 0.14218131 0.24647957 0.        ]]
0 0.8435993790626526 0.47535255551338196 0

ori-tar:[[0.25311843 0.69999886 0.21197063 0.51999992 0.        ]]
0 0.2531184256076813 0.6999988555908203 0.21197062730789185 0.5199999213218689

shape6_6_aug17
[[0.25311843 0.69999886 0.21197063 0.51999992 0.        ]]
ori-tar:[[0.53808248 0.67647022 0.11988748 0.17647056 0.        ]]
0 0.538082480430603 0.6764702200889587 0.11988747864961624 0.1764705628156662

shape6_6_aug18
[[0.53808248 0.67647022 0.11988748 0.17647056 0.        ]]
ori-tar:[[0.66609758 0.73876357 0.29109666 0.2191011  0.        ]]
0 0.6660975813865662 0.7387635707855225 0.29109665751457214 0.21910110116004944

shape6_6_aug19
[[0.66609758 0.73876357 0.29109666 0.2191011  0.        ]]
ori-tar:[[0.51180696 0.46123964 0.10035445 0.30232555 0.        ]]
0 0.5118069648742676 0.46123963594436646 0.10035444796085358 0.3023255467414856

shape6_6_aug20
[[0.51180696 0.46123964 0.10035445 0.30232555 0.        ]]
ori-tar:[[0.8229546  0.64087266 0.15124594 0.1547619  0.        ]]
0 0.8229545950889587 0.6408726572990417 0.151245

In [7]:
@interact(index=(0, len(trainset_crop)-1))

def show_sample(index=0):
    image, target, filename = trainset_crop[index]
#     image=image.permute(1,2,0).numpy()
    print(f"before cal target:\n{target}")
    img_H, img_W, _ = image.shape
    print(filename)
    print(image.shape)
#     print(image)

#     bboxes = target['boxes']
#     class_ids = target["labels"]
    
    ###
    bboxes = target[:, 0:4]
    class_ids = target[:, 4]
    ###
    bboxes[:, [0,2]] *= img_W
    bboxes[:, [1,3]] *= img_H
    
    print(bboxes)

    canvas = visualize(image, bboxes, class_ids)
    plt.figure(figsize=(6,6))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

# show_sample()

interactive(children=(IntSlider(value=0, description='index', max=1049), Output()), _dom_classes=('widget-inte…